In [2]:
import os
import sys
import torch
import random
import configs
import numpy as np
import transformers
import torch.nn as nn
from PIL import Image
import tensorflow as tf
from tqdm.auto import tqdm
from training_utils import val_loss_accuracy, get_test_accuracy

2023-11-16 12:46:25.035579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 12:46:25.035689: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 12:46:25.035727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 12:46:25.045468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 12:46:26.187567: W tensorflow/compiler/

In [3]:
configs.set_seed(42)
device = configs.set_device(2)

There are 8 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-80GB


In [4]:
import wandb

wandb.login()

wandb: Currently logged in as: semenov-andrei-v. Use `wandb login --relogin` to force relogin


True

In [5]:
import datasets

metric = datasets.load_metric("accuracy")

data

In [6]:
from tensorflow.keras.datasets import cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()


image_paths = []
labels = []

save_dir = 'cifar100_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for i in range(len(train_images)):
    image_path = os.path.join(save_dir, f"train_image_{i}.jpg")
    tf.keras.preprocessing.image.save_img(image_path, train_images[i])
    image_paths.append(image_path)
    labels.append(train_labels[i][0])


for i in range(len(test_images)):
    image_path = os.path.join(save_dir, f"test_image_{i}.jpg")
    tf.keras.preprocessing.image.save_img(image_path, test_images[i])
    image_paths.append(image_path)
    labels.append(test_labels[i][0])

print(len(train_images), "\n")
print(len(test_images), "\n")
print(len(image_paths))

50000 

10000 

60000


In [7]:
import dataset_utils

dataset = dataset_utils.CLIPDataset(list_image_path=image_paths, list_txt=labels)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [50000, 3000, 7000])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=dataset_utils.collate_fn, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=dataset_utils.collate_fn, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=dataset_utils.collate_fn, pin_memory=True)

assert len(dataset) == len(image_paths)
print("Dataset size: {}".format(len(dataset)), "\n")

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [50000, 3000, 7000])
print("Train set: {}".format(len(train_dataset)), "\n")
print("Validation set: {}".format(len(val_dataset)), "\n")
print("Test set: {}".format(len(test_dataset)))

Dataset size: 60000 

Train set: 50000 

Validation set: 3000 

Test set: 7000


In [8]:
def remove_prefixes(strings):
    prefixes = ['a', 'an', 'the']
    result = []

    for string in strings:
        words = string.split()
        if words[0].lower() in prefixes:
            result.append(' '.join(words[1:]))
        else:
            result.append(string)

    return result

with open("conceptnet_cifar10_filtered_new.txt", "r") as f:
    concepts = f.read().lower().split("\n")
    concepts = remove_prefixes(concepts)

In [8]:
from transformers import CLIPModel, CLIPProcessor, AutoTokenizer

train_loader_preprocessed = dataset_utils.preprocess_loader(train_loader, concepts)
val_loader_preprocessed = dataset_utils.preprocess_loader(val_loader, concepts)
test_loader_preprocessed = dataset_utils.preprocess_loader(test_loader, concepts)

100%|█████████████████████████████████████████| 219/219 [00:39<00:00,  5.51it/s]


model

In [12]:
class TuningCLIPhead(nn.Module):
    def __init__(self, concepts: list, model_name: str="openai/clip-vit-base-patch32"):
        super().__init__()
        self.clip = transformers.CLIPModel.from_pretrained(model_name)
        self.processor = transformers.CLIPProcessor.from_pretrained(model_name)
        for param in self.clip.parameters():
            param.requires_grad=False
        self.head = nn.Linear(len(concepts), 100, bias=False)

    def forward(self, **batch):
        out = self.clip(**batch).logits_per_image
        return self.head(out)

training

In [13]:
torch.cuda.empty_cache()

In [14]:
model = TuningCLIPhead(concepts=concepts)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Number of trainable parameters is: {}".format(sum(p.numel() for p in model.parameters() if p.requires_grad == True)))

for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

Number of trainable parameters is: 12000


In [13]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def val_loss_accuracy(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    val_losses = []
    for batch in tqdm(loader):
        inputs, labels = batch
        inputs = inputs.to(device)
        logits = model(**inputs).squeeze(0)
        targets = torch.tensor(labels, dtype=torch.long)
        loss = criterion(logits, targets.to(device))
        val_losses.append(loss.item())
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels)

    val_accuracy = metric.compute(predictions=all_preds, references=all_labels)
    avg_val_loss = sum(val_losses) / len(val_losses)
    
    return val_accuracy, avg_val_loss

@torch.no_grad()
@torch.cuda.amp.autocast()
def get_test_accuracy(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for batch in tqdm(loader):
        inputs, labels = batch
        inputs = inputs.to(device)
        logits = model(**inputs).squeeze(0)
        targets = torch.tensor(labels, dtype=torch.long)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels)
        
    test_accuracy = metric.compute(predictions=all_preds, references=all_labels)
    
    return test_accuracy

In [16]:
model.to(device)
run = wandb.init(project='cifar100-head', entity='semenov-andrei-v')

num_epochs = 60
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader_preprocessed, desc=f"Epoch {epoch + 1}/ {num_epochs}"):
        optimizer.zero_grad()
        
        inputs, labels = batch
        inputs = inputs.to(device)
        targets = torch.tensor(labels, dtype=torch.long)
        logits = model(**inputs).squeeze(0)
        
        loss = criterion(logits, targets.to(device))
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_accuracy, avg_val_loss = val_loss_accuracy(model, val_loader_preprocessed)

    wandb.log({"Loss/Train": loss.item(), 
               "Loss/Validation": avg_val_loss, 
               "Accuracy/Validation": val_accuracy['accuracy']}
             )

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {val_accuracy}")

model.eval()
test_accuracy = get_test_accuracy(model, train_loader_preprocessed)
wandb.log({"Accuracy/Test": test_accuracy['accuracy']})

run.finish()

Epoch 1/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 1/60, Loss: 5.463306427001953, Validation Accuracy: {'accuracy': 0.07133333333333333}


Epoch 2/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

...

Epoch 60/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 60/60, Loss: 2.9711647033691406, Validation Accuracy: {'accuracy': 0.429}


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy/Test,▁
Accuracy/Validation,▁▂▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████████
Loss/Train,█▆▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/Validation,█▆▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Accuracy/Test,0.45096
Accuracy/Validation,0.429
Loss/Train,2.97116
Loss/Validation,2.53374


## with cifar100 concepts only

In [9]:
with open("conceptnet_cifar100_filtered_new.txt", "r") as f:
    concepts = f.read().lower().split("\n")
    concepts = remove_prefixes(concepts)

In [10]:
len(concepts)

944

In [11]:
from transformers import CLIPModel, CLIPProcessor, AutoTokenizer

train_loader_preprocessed = dataset_utils.preprocess_loader(train_loader, concepts)
val_loader_preprocessed = dataset_utils.preprocess_loader(val_loader, concepts)
test_loader_preprocessed = dataset_utils.preprocess_loader(test_loader, concepts)

100%|█████████████████████████████████████████| 219/219 [00:44<00:00,  4.89it/s]


In [19]:
torch.cuda.empty_cache()

In [20]:
model = TuningCLIPhead(concepts=concepts)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Number of trainable parameters is: {}".format(sum(p.numel() for p in model.parameters() if p.requires_grad == True)))

for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

Number of trainable parameters is: 94400


In [21]:
model.to(device)
run = wandb.init(project='cifar100-head', entity='semenov-andrei-v')

num_epochs = 60
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader_preprocessed, desc=f"Epoch {epoch + 1}/ {num_epochs}"):
        optimizer.zero_grad()
        
        inputs, labels = batch
        inputs = inputs.to(device)
        targets = torch.tensor(labels, dtype=torch.long)
        logits = model(**inputs).squeeze(0)
        
        loss = criterion(logits, targets.to(device))
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_accuracy, avg_val_loss = val_loss_accuracy(model, val_loader_preprocessed)

    wandb.log({"Loss/Train": loss.item(), 
               "Loss/Validation": avg_val_loss, 
               "Accuracy/Validation": val_accuracy['accuracy']}
             )

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {val_accuracy}")

model.eval()
test_accuracy = get_test_accuracy(model, test_loader_preprocessed)
wandb.log({"Accuracy/Test": test_accuracy['accuracy']})

run.finish()

Epoch 1/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 1/60, Loss: 26.086469650268555, Validation Accuracy: {'accuracy': 0.11066666666666666}


Epoch 2/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

...

Epoch 60/ 60:   0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 60/60, Loss: 15.43360424041748, Validation Accuracy: {'accuracy': 0.376}


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy/Test,▁
Accuracy/Validation,▁▂▃▃▅▄▅▄▅▆▅▆▆▆▆▆▇▇▇▇▆▆▆▇▇▇▇▇▆▇▇▇▇▇▇█▇█▇▇
Loss/Train,█▂▃▄▆▄▅▄▃▃▇▆▄▃█▃▃▅▂▄▃▆▄▂▃▃▄▂▅▄▃▄▅▄▃▃▃▁▆▃
Loss/Validation,▆▆▆█▄▆▄▃▄▁▄▂▃▃▂▃▄▁▂▂▃▅▄▅▂▄▅▂█▅▄▁▃▅▄▂▆▁▆▅
Accuracy/Test,0.3884
Accuracy/Validation,0.376
Loss/Train,15.4336
Loss/Validation,13.64849
